# ImageJ Tutorials and Demo

Welcome to the ImageJ tutorial series. These notebooks offer a hands-on series of lessons for learning ImageJ.

* For a quick demo of what ImageJ can do, ___just scroll down___!

* To dive in to the tutorials, click the links below. If ImageJ is new to you, please try the "Using ImageJ" notebooks first.

* For a thorough academic overview of the ImageJ software stack, including its features, history, motivation and goals, see:
  > Rueden CT et al. "[ImageJ2: ImageJ for the next generation of scientific image data](https://doi.org/10.1186%2Fs12859-017-1934-z)." _BMC Bioinformatics_ __18:529__ (29 November 2017).
  
* Learn more about ImageJ at [imagej.net](https://imagej.net/). Learn more about Jupyter Notebook at [jupyter.org](https://jupyter.org/).
  
* See also [scikit-image tutorial notebooks](https://github.com/scikit-image/skimage-tutorials) and [SimpleITK tutorial notebooks](https://github.com/InsightSoftwareConsortium/SimpleITK-Notebooks).

Feedback is very welcome! Please share your ideas on the [Image.sc Forum](https://forum.image.sc/)!

<div class="row">
  <div class="col-md-12" style="border-top: solid 1px #888; margin-top: 20px;">
  </div>
</div>
<div class="row">
  <div class="col-md-6">
    <h2>Using ImageJ</h2>
    <ol>
      <li><a href="1-Using-ImageJ/1-Fundamentals.ipynb">Fundamentals of ImageJ</a></li>
      <li><a href="1-Using-ImageJ/2-ImageJ-Ops.ipynb">ImageJ Ops</a></li>
      <li><a href="1-Using-ImageJ/3-ImgLib2-Basics.ipynb">ImgLib2 Basics</li>
      <li><a href="1-Using-ImageJ/4-Tables.ipynb">Tables</a></li>
      <li><a href="1-Using-ImageJ/5-Mixed-World-ImageJ1.ipynb">Mixed World: Using ImageJ 1.x</a></li>
      <li><a href="1-Using-ImageJ/6-ImageJ-with-Python-Kernel.ipynb">ImageJ with Python Kernel</a></li>
      <li><a href="1-Using-ImageJ/7-Calling-Scripts-from-Scripts.ipynb">Calling Scripts from Scripts</a></li>
    </ol>
  </div>
  <div class="col-md-6">
    <h2>Extending ImageJ</h2>
    <ol>
      <li><a href="2-Extending-ImageJ/1-Input-Output.ipynb">IOPlugins - Data Input and Output</a></li>
      <li><a href="2-Extending-ImageJ/2-Ops.ipynb">Ops - Image Algorithms</a></li>
    </ol>
    <h2>Advanced Topics</h2>
    <ol>
      <li><a href="3-Advanced-Topics/1-SciJava-in-Detail.ipynb">SciJava in Detail</a></li>
      <li><a href="3-Advanced-Topics/2-ImgLib2-in-Detail.ipynb">ImgLib2 in Detail</a></li>
      <li><a href="3-Advanced-Topics/3-Using-Special-Ops.ipynb">Using Special Ops</a></li>
    </ol>
  </div>
</div>
<div class="row">
  <div class="col-md-12" style="border-top: solid 1px #888; margin-top: 20px;">
  </div>
</div>

## ImageJ Demo

First, we tell the notebook to download and install ImageJ from the remote repository.

In [ ]:
%classpath config resolver scijava.public https://maven.scijava.org/content/groups/public
%%classpath add mvn
net.imagej imagej 2.0.0-rc-71
net.imagej imagej-notebook 0.7.1

Now that ImageJ functionality is available to the notebook, we create an ImageJ gateway.

In [ ]:
ij = new net.imagej.ImageJ()
"ImageJ v${ij.getVersion()} is ready to go."

In [1]:
import imagej
ij = imagej.init();

In [3]:
ij.IJ

AttributeError: 'net.imagej.ImageJ' object has no attribute 'IJ'

For further details, see the [Fundamentals of ImageJ](1-Using-ImageJ/1-Fundamentals.ipynb) notebook.

### Load an image

You can open images from local files as well as remote URLs.

In [7]:
jimage = ij.io().open("https://imagej.net/images/lymp.tif")

In [8]:
image = ij.py.from_java(jimage)

In [ ]:
image

### Compute histogram

In [3]:
histogram = ij.op().image().histogram(image)

### N-dimensional crop

In [4]:
mandrill = ij.io().open("https://imagej.net/images/baboon.jpg")

[java.lang.Enum.toString] [INFO] Populating metadata


In [ ]:
import net.imglib2.FinalInterval
min = [85, 0, 110]
len = [5, 335, 1]
bounds = FinalInterval.createMinSize(min[0], len[0], min[1], len[1], min[2], len[2])
eyes = ij.op().transform().crop(mandrill, bounds, true)

### Type conversion

In [ ]:
eyes32 = ij.op().convert().float32(eyes)

In [ ]:
eyes.firstElement().getClass().getName()

In [ ]:
eyes32.firstElement().getClass().getName()

### Median filter with circular neighborhood

In [ ]:
import net.imglib2.algorithm.neighborhood.HyperSphereShape
median = ij.op().run("create.img", eyes32)
neighborhood = new HyperSphereShape(4)
ij.op().run("filter.median", median, eyes32, neighborhood)

### Difference of Gaussians formula

In [ ]:
dogFormula = "gauss(image, sigma1) - gauss(image, sigma2)"
dog = ij.op().eval(dogFormula, [
  "image": eyes32,
  "sigma1": [20, 20],
  "sigma2": [4, 4]
])

### Grayscale morphology operators

In [ ]:
topHat = ij.op().morphology().topHat(eyes, [neighborhood])

In [ ]:
blackTopHat = ij.op().morphology().blackTopHat(eyes, [neighborhood])

### Display multiple images at once

In [ ]:
ij.notebook().display(["median":median, "topHat":topHat, "blackTopHat":blackTopHat])

In [ ]:
ij.notebook().display([["median":median, "topHat":topHat, "blackTopHat":blackTopHat]])

### Fourier transform with lowpass filter

Define a lowpass filtering function, operating in the Fourier domain:

In [ ]:
import net.imglib2.util.Util

lowpass = { fft, radius ->
  // Declare an array to hold the current position of the cursor.
  pos = new long[fft.numDimensions()]

  // Define origin as 0,0.
  long[] origin = [0, 0]

  // Define a 2nd 'origin' at bottom left of image.
  // This is a bit of a hack. We want to draw a circle around the origin,
  // since the origin is at 0,0 - the circle will 'reflect' to the bottom.
  long[] origin2 = [0, fft.dimension(1)]

  // Loop through all pixels.
  cursor = fft.localizingCursor()
  while (cursor.hasNext()) {
    cursor.fwd()
    cursor.localize(pos)

    // Calculate distance from 0,0 and bottom left corner
    // (so we can form the reflected semi-circle).
    dist = Util.distance(origin, pos)
    dist2 = Util.distance(origin2, pos)

    // If distance is above radius (cutoff frequency) set value of FFT to zero.
    if (dist > radius && dist2 > radius)
      cursor.get().setZero()
  }
}

Perform a fast Fourier transform (FFT) on the image, run the lowpass filter, then invert the FFT:

In [ ]:
import net.imglib2.type.numeric.real.FloatType

// Perform fft of the input.
fft = ij.op().filter().fft(image)

// Filter it.
lowpass(fft, radius=10)

// Reverse the FFT.
inverse = ij.op().run("create.img", image, new FloatType())
ij.op().filter().ifft(inverse, fft)

// Display the result.
ij.notebook().display([["image":image, "lowpass":inverse]])